In [78]:
import numpy as np
import pandas as pd
import json

### How do we predict what rating a user is going to give?
1) Business side:
    - location
    - type
    - hours
    - parking availability
    - ambience
    - takeout?
2) User side:
    - how long on yelp
    - elite status
    - review count
    - useful/funny/cool
    - home

# TODO
    - Matthew
        - isolate restaurants in business_df
        - preprocessing: marking reviews as good or bad
    - feature extraction
        - hometown
        - number of tips by user

    - other preprocessing
        - yelpin since: timestamp today - yelping since
        - elite status: how many years have they been elite
        - only consider business with certain number of reviews
        - only consider "active users"

# Next
    - what happens if we remove "inactive" users

        

In [79]:
business_df = pd.read_csv("sample_business.csv")
checkin_df = pd.read_csv("sample_checkin.csv")
review_df = pd.read_csv("sample_review.csv")
tip_df = pd.read_csv("sample_tip.csv")
user_df = pd.read_csv("sample_user.csv")

In [95]:
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,JlLh80kYzjVPcINyAezPKQ,Santa Cruz Animal Clinic,431 W Robertson St,Brandon,FL,33511,27.935510,-82.291042,3.5,23,1,NaN,"Pets, Pet Services, Veterinarians","{'Monday': '7:30-17:30', 'Tuesday': '7:30-17:3..."
1,2NOCur2ahFs3ypwT3VPNQQ,Lion's Choice,9621 Manchester Rd,St. Louis,MO,63119,38.609162,-90.366873,4.0,13,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Fast Food, Restaurants, American (Traditional)...","{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'..."
2,6neDipsScU1gtNXKHb37ZQ,Old Navy,2440 Chemical Rd,Plymouth Meeting,PA,19462,40.106164,-75.292746,3.5,8,1,"{'BikeParking': 'False', 'BusinessParking': ""{...","Fashion, Women's Clothing, Men's Clothing, Chi...","{'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'..."
3,Q5PYc597t-2Usp905WpDWg,"Mark Berg, DDS",NaN,Saint Louis,MO,63141,38.662494,-90.478549,3.0,18,1,{'ByAppointmentOnly': 'True'},"Oral Surgeons, Dentists, Health & Medical, End...",NaN
4,AtPTHg-NaRK93kFLEBDyOA,TheraTub Walk-In Tub,NaN,Brandon,FL,33511,27.908259,-82.290915,3.5,5,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Shopping, Kitchen & Bath, Home Services, Home ...","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,8D10wCHiMvF8qfl8N-jE_A,Ardmore Grooming station,7 Anderson Ave,Ardmore,PA,19003,40.008321,-75.290824,3.5,18,1,NaN,"Pet Services, Pet Groomers, Pets",NaN
39996,9BRogQ1ZU-agQCPG-a4zIg,Groom,1324 Locust St,Philadelphia,PA,19107,39.948062,-75.163653,4.5,172,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Hair Salons, Barbers, Beauty & Spas","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
39997,o_9hBDxSVk0urSG9DhSF8A,Airport Motors,5950 Olney St,Goleta,CA,93117,34.429898,-119.829963,4.0,63,1,{'BusinessAcceptsCreditCards': 'True'},"Auto Repair, Automotive","{'Monday': '7:30-17:0', 'Tuesday': '7:30-17:0'..."
39998,Z0U0sPwogMRdbhNZzx2wgw,The Boiling Pot,319 Market St,Philadelphia,PA,19106,39.950390,-75.146338,3.5,40,0,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Seafood, Asian Fusion, Restaurants, Cajun/Creole","{'Monday': '14:0-20:0', 'Tuesday': '14:0-20:0'..."


In [80]:
business_df.drop(columns = 'Unnamed: 0', inplace = True)
business_loc = business_df.iloc[:, [0,3, 4, 5, 6, 7]]

In [81]:
user_useful = user_df.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
user_useful.set_index('user_id', inplace = True)

In [82]:
review_df.drop(columns = 'Unnamed: 0', inplace = True)

In [83]:
rJoinU = review_df.join(user_useful, on = 'user_id', lsuffix='_Rev', rsuffix = '_User')

In [84]:
rJoinU.dropna(inplace = True)

In [85]:
business_loc.set_index('business_id', inplace= True)

In [86]:
rJUJBloc = rJoinU.join(business_loc, on='business_id', lsuffix='_rJU', rsuffix='_bloc')

In [87]:
rJUJBloc.dropna(inplace= True)

In [126]:
rJUJBloc

,review_id,user_id,business_id,stars,useful_Rev,funny_Rev,cool_Rev,text,date,name,...,cool_User,elite,friends,fans,average_stars,city,state,postal_code,latitude,longitude
350,tWZCoR6UQL_BSUSQpuF1SQ,HjCfLjX6pbL-RSVuxRnHGA,2JBFrR3KzCfBaFl5sPwSWg,5,0,0,0,I had a really good experience here. I had bee...,2017-05-10 06:59:24,Trish,...,122.0,"2016,2017,2018,2019,20,20,2021","hgbCQJ0xZVNBPxxeJOLybA, 9Und8nOzcBD84GOGbjW45A...",15.0,4.19,Tucson,AZ,85715,32.249197,-110.850244
669,bs2j-w4rk-DFX7RaI5E31g,wTdEJv1VYUdlZ0_fS_vT5A,p0UtMDuBcPkTmP9x3SEZlw,5,3,1,1,Christine was just amazing! Her knowledge abou...,2010-11-06 03:24:35,Paulina,...,62.0,"2010,2011,2012","DAeC-Y66n-jNMJJw4oAfNQ, qpFCY_wj8_G-HOV8MxIaHw...",11.0,4.06,New Orleans,LA,70115,29.920996,-90.117020
1759,pKf7zgp8hNWRf0Q8kIXtmQ,tBI1rbIioJ7o1_00OMXaCw,o7cD_sDA-Evj3Oh0zeH9nQ,3,1,0,2,Average food and just average quality. When I ...,2015-10-24 03:50:25,Perfecto,...,3377.0,"2015,2016,2017,2018,2019,20,20","9-4ggnCbnH6kHr_cEkj47g, R0_RUi0dGRe_yZ5JLZgvRQ...",95.0,4.05,Reno,NV,89511,39.477210,-119.764903
2485,WbgQBZZz8fDGDa2aHBkh-A,Y4paTTEMd6sNGSHVVcKnRg,J7_B_oDTGbFm4fe0Uy--lQ,4,0,0,0,Employees are very nice\n\nAtmosphere is chic ...,2013-09-29 05:39:00,Libby,...,31.0,"2016,2017","rGTbmuoo29GGlu2pAxeVyw, sKo2PFEGoDPfORw-_kH1LA...",11.0,3.86,Indianapolis,IN,46205,39.846614,-86.146095
5454,Ntz_TpNweXGUEA4roylGrQ,qRBTAUOEWrBY9N1JszH1ow,_49FsmTfruZSTjxU4OwOsg,5,1,0,1,This place was one of my favorite meals during...,2016-01-09 19:34:14,Swetha,...,550.0,"2009,2010,2011,2012,2013,2014,2015,2016,2017","ppumt9yLAoYZJLWN2Df-jQ, OOwv1ocICoa5Ulggtfrerg...",41.0,4.08,Nashville,TN,37212,36.143655,-86.813522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38377,witJoxglZ8K54XZ6ITNHfA,vl3qiO6v4RbFJBpxVREJPw,2Q1R2OhBbAQ581vK_r7NhA,5,0,0,0,"Lighted stairway leads to the relaxing , calm ...",2015-01-28 02:26:29,Jennifer,...,31.0,"2015,2016","25VrxgIm9K1hEOl2-SAwJw, GbCn6Fa9MjUvQ9aJR1UnaA...",3.0,4.39,Willow Grove,PA,19090,40.156669,-75.121837
38685,5qAYZLUt7nBajAb2joUAdA,zxuxd6Hz2tKcpgZ71dYEcw,ddibKJk-XuUhom1uI754WA,4,1,0,0,"Panda hut makes pretty good Chinese food, argu...",2013-12-14 02:59:57,Russell,...,260.0,"2014,2015,2016","_mensKWwcLtotC_UR6R75w, kkDUDXchYnv1cUbvhboh7Q...",16.0,3.90,Edmonton,AB,T6J 7B5,53.453330,-113.508505
39260,m8EI4neOxkag_fx0ak0tuw,mKBl4fAqTfNts7B78aOPVg,ts28to44UDqPxIKIlt68yQ,3,2,2,0,Time killer place for me is all I see.\n\nAfte...,2014-09-30 05:35:14,Dave,...,1770.0,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","TrL_9PA5LSDXgCYU8I7NCg, vnl8aTXiLC08mVtG5Tu_xw...",76.0,3.68,Saint Louis,MO,63110,38.635676,-90.265144
39685,MwY0gy8ZT-YE9zvdHb_kew,Ttn1RtATzZBtWsx6MoPGqQ,9XpU6r6ATXSWYfbU7kibaQ,5,2,0,0,I've been a customer of their Suburban Station...,2016-01-08 17:22:56,David,...,322.0,"2015,2016,2017,2018,2019,20,20,2021","0K9H88eoVeONhxIbj-ywlw, z7kBLlOXJAqZxRuKZLz16w...",20.0,4.49,Philadelphia,PA,19103,39.954201,-75.167466


In [136]:
hometown = rJUJBloc.groupby(['user_id'])['city'].agg(lambda x: pd.Series.mode(x)[0]).to_frame()

In [137]:
hometown

,city
user_id,
2ZAGNRs2kmUk1H2BOW_3OA,Philadelphia
2jHXlagPsLtmRi4Nsr49LA,Tucson
2pDWLhRkZa67GvNu0DWQtA,Philadelphia
4rrK8opMym8ZsiaasWYpww,Philadelphia
5nd9UFG5AY_PEsUYZoCKJg,Philadelphia
...,...
v4OSDIDxgwqpa4bvH4QIqg,New Orleans
vl3qiO6v4RbFJBpxVREJPw,Willow Grove
wTdEJv1VYUdlZ0_fS_vT5A,New Orleans


In [138]:
data = rJUJBloc.merge(hometown, how = 'left', on="user_id", suffixes=('_x', '_y'))

In [142]:
data['is_home'] = (data['city_x'] == data['city_y']).astype(int)

84

In [59]:
test = pd.DataFrame({'a': ['friends', 'family', 'enemies'],
                    'b': ['friends', 'family', 'enemy']})

In [60]:
test

,a,b
0,friends,friends
1,family,family
2,enemies,enemy


In [61]:
test['equ'] = test['a'] == test['b']

In [74]:
test

,a,b,equ
0,friends,friends,True
1,family,family,True
2,enemies,enemy,False
